<a href="https://colab.research.google.com/github/ayushig2004/project4/blob/main/ai'_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pgmpy numpy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 17.3 MB/s eta 0:00:00


In [ ]:
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination

# Step 1: Define the structure of the Bayesian Network
# In this example, we'll model the relationship between a disease and its symptoms (fever, cough, fatigue).
model = BayesianNetwork([('Disease', 'Fever'),
                         ('Disease', 'Cough'),
                         ('Disease', 'Fatigue')])

# Step 2: Define the CPDs (Conditional Probability Distributions)

# CPD for Disease: P(Disease)
# We assume a 30% chance of having the disease and 70% chance of not having it
cpd_disease = TabularCPD(variable='Disease', variable_card=2, values=[[0.7], [0.3]])  # No Disease, Disease

# CPD for Fever given Disease
# If Disease = 1, P(Fever=Yes) = 0.8; If Disease = 0, P(Fever=Yes) = 0.2
cpd_fever = TabularCPD(variable='Fever', variable_card=2,
                       values=[[0.8, 0.2],  # P(Fever=No | Disease=No), P(Fever=No | Disease=Yes)
                               [0.2, 0.8]], # P(Fever=Yes | Disease=No), P(Fever=Yes | Disease=Yes)
                       evidence=['Disease'], evidence_card=[2])

# CPD for Cough given Disease
# If Disease = 1, P(Cough=Yes) = 0.7; If Disease = 0, P(Cough=Yes) = 0.3
cpd_cough = TabularCPD(variable='Cough', variable_card=2,
                       values=[[0.7, 0.3],  # P(Cough=No | Disease=No), P(Cough=No | Disease=Yes)
                               [0.3, 0.7]], # P(Cough=Yes | Disease=No), P(Cough=Yes | Disease=Yes)
                       evidence=['Disease'], evidence_card=[2])
# CPD for Fatigue given Disease
# If Disease = 1, P(Fatigue=Yes) = 0.6; If Disease = 0, P(Fatigue=Yes) = 0.2
cpd_fatigue = TabularCPD(variable='Fatigue', variable_card=2,
                         values=[[0.6, 0.2],  # P(Fatigue=No | Disease=No), P(Fatigue=No | Disease=Yes)
                                 [0.4, 0.8]], # P(Fatigue=Yes | Disease=No), P(Fatigue=Yes | Disease=Yes)
                         evidence=['Disease'], evidence_card=[2])

# Step 3: Add the CPDs to the model
model.add_cpds(cpd_disease, cpd_fever, cpd_cough, cpd_fatigue)
# Step 4: Verify the model structure
assert model.check_model()

# Step 5: Perform Exact Inference using Variable Elimination
# Initialize the inference engine
inference = VariableElimination(model)

# Step 6: Query the network with evidence
# Query 1: Probability of Disease given that the patient has Fever=True and Cough=True
result_1 = inference.query(variables=['Disease'], evidence={'Fever': 1, 'Cough': 1})
print("Query 1: Probability of Disease given Fever=True and Cough=True:\n", result_1)

# Query 2: Probability of Disease given that the patient has Fatigue=True
result_2 = inference.query(variables=['Disease'], evidence={'Fatigue': 1})
print("\nQuery 2: Probability of Disease given Fatigue=True:\n", result_2)

# Query 3: Probability of Disease with no evidence (prior probability)
result_3 = inference.query(variables=['Disease'])
print("\nQuery 3: Probability of Disease with no evidence:\n",result_3)

Query 1: Probability of Disease given Fever=True and Cough=True:
 +------------+----------------+
| Disease    |   phi(Disease) |
+============+================+
| Disease(0) |         0.2000 |
+------------+----------------+
| Disease(1) |         0.8000 |
+------------+----------------+

Query 2: Probability of Disease given Fatigue=True:
 +------------+----------------+
| Disease    |   phi(Disease) |
+============+================+
| Disease(0) |         0.5385 |
+------------+----------------+
| Disease(1) |         0.4615 |
+------------+----------------+

Query 3: Probability of Disease with no evidence:
 +------------+----------------+
| Disease    |   phi(Disease) |
+============+================+
| Disease(0) |         0.7000 |
+------------+----------------+
| Disease(1) |         0.3000 |
+------------+----------------+
